In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive подключен!")

Mounted at /content/drive
✅ Google Drive подключен!


In [ ]:
import zipfile
import os

ZIP_FILE_PATH = '/content/drive/MyDrive/Resampled_HAM10000.zip'
OUTPUT_DIR = 'dataset_resampled'

if not os.path.exists(ZIP_FILE_PATH):
    print(f"ОШИКА: Файл не найден по пути {ZIP_FILE_PATH}")
    print("Убедись, что ты загрузил 'Resampled_HAM10000.zip' в основную папку 'My Drive'.")
else:
    print(f"Файл {ZIP_FILE_PATH} найден. Начинаем распаковку...")

    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)

    with zipfile.ZipFile(ZIP_FILE_PATH, 'r') as zip_ref:
        zip_ref.extractall(OUTPUT_DIR)

    print(f"\n---  Успешно Распаковано! ---")
    print(f"Все сбалансированные данные теперь в папке: '{OUTPUT_DIR}'")

Файл /content/drive/MyDrive/Resampled_HAM10000.zip найден. Начинаем распаковку...

---  Успешно Распаковано! ---
Все сбалансированные данные теперь в папке: 'dataset_resampled'


In [ ]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"yerkh1","key":"670f260d61e76c4f89a16083d97d93bb"}'}

In [ ]:

!mkdir -p ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

print("API Токен настроен.")

!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

API Токен настроен.
Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
User cancelled operation


In [ ]:
import zipfile


with zipfile.ZipFile('skin-cancer-mnist-ham10000.zip', 'r') as zip_ref:
    zip_ref.extractall('ham10000_dataset')

print("Готово! Датасет распакован в папку 'ham10000_dataset'")

✅ Готово! Датасет распакован в папку 'ham10000_dataset'


In [ ]:
from google.colab import files

print("Пожалуйста, выберите файл 'fitzpatrick17k.csv' для загрузки...")
uploaded = files.upload()

if 'fitzpatrick17k.csv' in uploaded:
    print("\n✅ Файл 'fitzpatrick17k.csv' успешно загружен!")
else:
    print("\n⚠️ Ошибка: Файл не был загружен или имеет другое имя.")

Пожалуйста, выберите файл 'fitzpatrick17k.csv' для загрузки...


Saving fitzpatrick17k.csv to fitzpatrick17k.csv

✅ Файл 'fitzpatrick17k.csv' успешно загружен!


In [ ]:
import pandas as pd
import os
import requests
from tqdm.notebook import tqdm

print("--- Запуск скрипта для скачивания изображений рака на темной коже ---")

CSV_PATH = 'fitzpatrick17k.csv'
OUTPUT_DIR = 'dark_skin_cancer_images'

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Папка '{OUTPUT_DIR}' создана.")

try:
    df = pd.read_csv(CSV_PATH)
except FileNotFoundError:
    print(f"ОШИБКА: Файл '{CSV_PATH}' не найден. Убедись, что Шаг 1 прошел успешно.")
    raise

cancer_labels = [
    'melanoma',
    'superficial spreading melanoma ssm',
    'basal cell carcinoma',
    'squamous cell carcinoma',
    'malignant melanoma'
]
dark_phototypes = [5, 6]

filtered_df = df[
    df['label'].isin(cancer_labels) &
    df['fitzpatrick_scale'].isin(dark_phototypes)
].copy()

print(f"Найдено {len(filtered_df)} изображений, соответствующих критериям.")


download_count = 0
error_count = 0

for index, row in tqdm(filtered_df.iterrows(), total=filtered_df.shape[0], desc="Скачивание"):

    image_url = row['url']
    image_label = str(row['label']).replace(" ", "_")
    image_fst = row['fitzpatrick_scale']
    image_md5 = row['md5hash']

    filename = f"fst_{image_fst}_{image_label}_{image_md5}.jpg"
    output_path = os.path.join(OUTPUT_DIR, filename)

    if pd.isna(image_url):
        error_count += 1
        continue

    try:

        response = requests.get(image_url, timeout=10)
        response.raise_for_status()

        with open(output_path, 'wb') as f:
            f.write(response.content)
        download_count += 1

    except requests.exceptions.RequestException as e:
        error_count += 1

print("\n---  Скачивание Завершено! ---")
print(f"Успешно скачано: {download_count} изображений")
print(f"Ошибок/Пропущено: {error_count}")
print(f"Все изображения сохранены в папке: '{OUTPUT_DIR}'")

--- Запуск скрипта для скачивания изображений рака на темной коже ---
Папка 'dark_skin_cancer_images' создана.
Найдено 137 изображений, соответствующих критериям.


Скачивание:   0%|          | 0/137 [00:00<?, ?it/s]


---  Скачивание Завершено! ---
Успешно скачано: 135 изображений
Ошибок/Пропущено: 2
Все изображения сохранены в папке: 'dark_skin_cancer_images'


In [ ]:
!pip install albumentations

In [ ]:
import albumentations as A
import cv2
import os
from tqdm.notebook import tqdm
import random

print("Библиотеки Albumentations и OpenCV импортированы.")

INPUT_DIR = 'dark_skin_cancer_images'
OUTPUT_DIR = 'augmented_dark_skin_images'
TARGET_IMAGES_PER_GROUP = 5000

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Папка '{OUTPUT_DIR}' создана.")

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),

    A.Rotate(limit=30, p=0.7),

    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.8),

    A.GaussianBlur(blur_limit=(3, 7), p=0.3),

    A.ElasticTransform(p=0.3),
])

try:
    image_list = [f for f in os.listdir(INPUT_DIR) if f.endswith('.jpg')]
    if not image_list:
        print(f"ОШИБКА: Папка '{INPUT_DIR}' пуста. Убедись, что Шаг 1 прошел успешно.")
        raise FileNotFoundError

    print(f"Найдено {len(image_list)} исходных изображений для аугментации.")

except FileNotFoundError:
    print(f"ОШИБКА: Папка '{INPUT_DIR}' не найдена.")


if image_list:

    num_augmentations_per_image = TARGET_IMAGES_PER_GROUP // len(image_list)

    print(f"Цель: {TARGET_IMAGES_PER_GROUP} изображений.")
    print(f"Каждое из {len(image_list)} исходных изображений будет аугментировано ~{num_augmentations_per_image} раз.")

    total_generated = 0

    for img_name in tqdm(image_list, desc="Обработка исходных изображений"):
        img_path = os.path.join(INPUT_DIR, img_name)

        image = cv2.imread(img_path)

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        for i in range(num_augmentations_per_image):

            augmented_image_data = transform(image=image)
            augmented_image = augmented_image_data['image']

            base_name, ext = os.path.splitext(img_name)
            new_filename = f"{base_name}_aug_{i}{ext}"
            output_path = os.path.join(OUTPUT_DIR, new_filename)

            augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)

            cv2.imwrite(output_path, augmented_image_bgr)
            total_generated += 1

    print(f"\n---  Шаг 2 Завершен! ---")
    print(f"Сгенерировано и сохранено {total_generated} новых изображений.")
    print(f"Они находятся в папке: '{OUTPUT_DIR}'")

Библиотеки Albumentations и OpenCV импортированы.
Папка 'augmented_dark_skin_images' создана.
Найдено 135 исходных изображений для аугментации.
Цель: 5000 изображений.
Каждое из 135 исходных изображений будет аугментировано ~37 раз.


Обработка исходных изображений:   0%|          | 0/135 [00:00<?, ?it/s]


---  Шаг 2 Завершен! ---
Сгенерировано и сохранено 4995 новых изображений.
Они находятся в папке: 'augmented_dark_skin_images'


In [ ]:
import pandas as pd
import os
import numpy as np

print("--- Шаг 1 (v3, Финальный): Создание СБАЛАНСИРОВАННОГО 'Мастер-Списка' ---")

# --- 1. Настройка Папок (ИСПРАВЛЕНИЕ ЗДЕСЬ) ---
BASE_DIR = 'ham10000_dataset'

# --- !!! ИСПРАВЛЕНИЕ: Указываем на ОРИГИНАЛЬНЫЙ metadata.csv !!! ---
METADATA_PATH = os.path.join(BASE_DIR, 'HAM10000_metadata.csv')
# --- !!! --------------------------------------------- !!! ---

IMAGE_DIR_PART1 = os.path.join(BASE_DIR, 'HAM10000_images_part_1')
IMAGE_DIR_PART2 = os.path.join(BASE_DIR, 'HAM10000_images_part_2')
AUGMENTED_DIR = 'augmented_dark_skin_images'

# --- 2. Подготовка данных HAM10000 ---
image_path_dict = {}
for img_name in os.listdir(IMAGE_DIR_PART1):
    if img_name.endswith('.jpg'):
        image_path_dict[os.path.splitext(img_name)[0]] = os.path.join(IMAGE_DIR_PART1, img_name)
for img_name in os.listdir(IMAGE_DIR_PART2):
    if img_name.endswith('.jpg'):
        image_path_dict[os.path.splitext(img_name)[0]] = os.path.join(IMAGE_DIR_PART2, img_name)

# Загружаем из ОРИГИНАЛЬНОГО CSV
try:
    df_ham = pd.read_csv(METADATA_PATH, usecols=['image_id', 'dx'])
except FileNotFoundError:
    print(f"ОШИБКА: Файл {METADATA_PATH} не найден. Убедись, что 'ham10000_dataset' распакована.")
    raise

df_ham['image_path'] = df_ham['image_id'].map(image_path_dict)
df_ham = df_ham.rename(columns={'dx': 'label'})

# --- 3. Подготовка Аугментированных данных (без изменений) ---
label_map = {
    'melanoma': 'mel',
    'superficial_spreading_melanoma_ssm': 'mel',
    'malignant_melanoma': 'mel',
    'basal_cell_carcinoma': 'bcc',
    'squamous_cell_carcinoma': 'akiec'
}
aug_image_paths = []
aug_image_labels = []
for img_name in os.listdir(AUGMENTED_DIR):
    if img_name.endswith('.jpg'):
        aug_image_paths.append(os.path.join(AUGMENTED_DIR, img_name))
        found_label = None
        for key, value in label_map.items():
            if key in img_name:
                found_label = value
                break
        aug_image_labels.append(found_label if found_label else 'unknown')

df_aug = pd.DataFrame({
    'image_path': aug_image_paths,
    'label': aug_image_labels
})

# --- 4. АГРЕССИВНАЯ БАЛАНСИРОВКА ('Resampling') ---
print("Применение АГРЕССИВНОЙ балансировки (Undersampling) для класса 'nv'...")

df_ham_nv = df_ham[df_ham['label'] == 'nv']
df_ham_others = df_ham[df_ham['label'] != 'nv']

# Устанавливаем лимит 'nv' равным другим большим классам
NV_LIMIT = 2700
df_ham_nv_sampled = df_ham_nv.sample(n=NV_LIMIT, random_state=42)
print(f"Оставили {len(df_ham_nv_sampled)} изображений 'nv' (вместо {len(df_ham_nv)}).")

# --- 5. Объединение ---
all_data_df = pd.concat([df_ham_nv_sampled, df_ham_others, df_aug])
all_data_df = all_data_df[all_data_df['label'] != 'unknown']
all_data_df = all_data_df.dropna(subset=['image_path'])
all_data_df = all_data_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n--- ✅ СБАЛАНСИРОВАННЫЙ 'Мастер-Список' (v3) создан! ---")
print(f"Всего изображений для обучения: {len(all_data_df)}")
print("\nФинальное Распределение по классам:")
print(all_data_df['label'].value_counts())

--- Шаг 1 (v3, Финальный): Создание СБАЛАНСИРОВАННОГО 'Мастер-Списка' ---
Применение АГРЕССИВНОЙ балансировки (Undersampling) для класса 'nv'...
Оставили 2700 изображений 'nv' (вместо 6705).

--- ✅ СБАЛАНСИРОВАННЫЙ 'Мастер-Список' (v3) создан! ---
Всего изображений для обучения: 11005

Финальное Распределение по классам:
label
nv       2700
akiec    2658
mel      2630
bcc      1661
bkl      1099
vasc      142
df        115
Name: count, dtype: int64


In [ ]:
import pandas as pd
import os
import numpy as np

print("--- Шаг 1 (v3, Финальный): Создание СБАЛАНСИРОВАННОГО 'Мастер-Списка' ---")

# --- 1. Настройка Папок (ИСПРАВЛЕНИЕ ЗДЕСЬ) ---
BASE_DIR = 'ham10000_dataset'

# --- !!! ИСПРАВЛЕНИЕ: Указываем на ОРИГИНАЛЬНЫЙ metadata.csv !!! ---
METADATA_PATH = os.path.join(BASE_DIR, 'HAM10000_metadata.csv')
# --- !!! --------------------------------------------- !!! ---

IMAGE_DIR_PART1 = os.path.join(BASE_DIR, 'HAM10000_images_part_1')
IMAGE_DIR_PART2 = os.path.join(BASE_DIR, 'HAM10000_images_part_2')
AUGMENTED_DIR = 'augmented_dark_skin_images'

# --- 2. Подготовка данных HAM10000 ---
image_path_dict = {}
for img_name in os.listdir(IMAGE_DIR_PART1):
    if img_name.endswith('.jpg'):
        image_path_dict[os.path.splitext(img_name)[0]] = os.path.join(IMAGE_DIR_PART1, img_name)
for img_name in os.listdir(IMAGE_DIR_PART2):
    if img_name.endswith('.jpg'):
        image_path_dict[os.path.splitext(img_name)[0]] = os.path.join(IMAGE_DIR_PART2, img_name)

# Загружаем из ОРИГИНАЛЬНОГО CSV
try:
    df_ham = pd.read_csv(METADATA_PATH, usecols=['image_id', 'dx'])
except FileNotFoundError:
    print(f"ОШИБКА: Файл {METADATA_PATH} не найден. Убедись, что 'ham10000_dataset' распакована.")
    raise

df_ham['image_path'] = df_ham['image_id'].map(image_path_dict)
df_ham = df_ham.rename(columns={'dx': 'label'})

# --- 3. Подготовка Аугментированных данных (без изменений) ---
label_map = {
    'melanoma': 'mel',
    'superficial_spreading_melanoma_ssm': 'mel',
    'malignant_melanoma': 'mel',
    'basal_cell_carcinoma': 'bcc',
    'squamous_cell_carcinoma': 'akiec'
}
aug_image_paths = []
aug_image_labels = []
for img_name in os.listdir(AUGMENTED_DIR):
    if img_name.endswith('.jpg'):
        aug_image_paths.append(os.path.join(AUGMENTED_DIR, img_name))
        found_label = None
        for key, value in label_map.items():
            if key in img_name:
                found_label = value
                break
        aug_image_labels.append(found_label if found_label else 'unknown')

df_aug = pd.DataFrame({
    'image_path': aug_image_paths,
    'label': aug_image_labels
})

# --- 4. АГРЕССИВНАЯ БАЛАНСИРОВКА ('Resampling') ---
print("Применение АГРЕССИВНОЙ балансировки (Undersampling) для класса 'nv'...")

df_ham_nv = df_ham[df_ham['label'] == 'nv']
df_ham_others = df_ham[df_ham['label'] != 'nv']

# Устанавливаем лимит 'nv' равным другим большим классам
NV_LIMIT = 2700
df_ham_nv_sampled = df_ham_nv.sample(n=NV_LIMIT, random_state=42)
print(f"Оставили {len(df_ham_nv_sampled)} изображений 'nv' (вместо {len(df_ham_nv)}).")

# --- 5. Объединение ---
all_data_df = pd.concat([df_ham_nv_sampled, df_ham_others, df_aug])
all_data_df = all_data_df[all_data_df['label'] != 'unknown']
all_data_df = all_data_df.dropna(subset=['image_path'])
all_data_df = all_data_df.sample(frac=1, random_state=42).reset_index(drop=True)

print(f"\n--- ✅ СБАЛАНСИРОВАННЫЙ 'Мастер-Список' (v3) создан! ---")
print(f"Всего изображений для обучения: {len(all_data_df)}")
print("\nФинальное Распределение по классам:")
print(all_data_df['label'].value_counts())

--- Шаг 1 (v3, Финальный): Создание СБАЛАНСИРОВАННОГО 'Мастер-Списка' ---
Применение АГРЕССИВНОЙ балансировки (Undersampling) для класса 'nv'...
Оставили 2700 изображений 'nv' (вместо 6705).

--- ✅ СБАЛАНСИРОВАННЫЙ 'Мастер-Список' (v3) создан! ---
Всего изображений для обучения: 11005

Финальное Распределение по классам:
label
nv       2700
akiec    2658
mel      2630
bcc      1661
bkl      1099
vasc      142
df        115
Name: count, dtype: int64


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

print("\n--- Шаг 2 (Повторный запуск): Создание tf.data Pipeline ---")

IMG_SIZE = 224
BATCH_SIZE = 32

label_encoder = LabelEncoder()
all_data_df['label_encoded'] = label_encoder.fit_transform(all_data_df['label'])
NUM_CLASSES = len(label_encoder.classes_)

print(f"Классы преобразованы в числа. Всего классов: {NUM_CLASSES}")
print(f"Классы: {list(label_encoder.classes_)}")

train_df, val_df = train_test_split(
    all_data_df,
    test_size=0.2,
    random_state=42,
    stratify=all_data_df['label']
)
print(f"Данные разделены: {len(train_df)} (Train) / {len(val_df)} (Val)")

def load_image_and_label(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = img / 255.0
    return img, label

train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_df['image_path'].values, train_df['label_encoded'].values)
)
train_dataset = train_dataset.map(load_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=1000)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices(
    (val_df['image_path'].values, val_df['label_encoded'].values)
)
val_dataset = val_dataset.map(load_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

print("\n---  СБАЛАНСИРОВАННЫЕ 'train_dataset' и 'val_dataset' созданы! ---")


--- Шаг 2 (Повторный запуск): Создание tf.data Pipeline ---
Классы преобразованы в числа. Всего классов: 7
Классы: ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
Данные разделены: 8804 (Train) / 2201 (Val)

---  СБАЛАНСИРОВАННЫЕ 'train_dataset' и 'val_dataset' созданы! ---


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetV2B0
from tensorflow.keras.optimizers import Adam

print("--- Шаг 3 (ИСПРАВЛЕННЫЙ, Low LR): Построение Unimodal Модели ---")

try:
    input_shape = (IMG_SIZE, IMG_SIZE, 3)
    NUM_CLASSES
    print(f"Форма входа: {input_shape}, Классов: {NUM_CLASSES}")
except NameError:
    print("ОШИБКА: 'IMG_SIZE' или 'NUM_CLASSES' не найдены. Запусти Шаг 2.")
    raise
inputs = Input(shape=input_shape)

x = Rescaling(1./255)(inputs)

base_model = EfficientNetV2B0(include_top=False, weights='imagenet', input_tensor=x)
base_model.trainable = False
print("Базовая модель EfficientNetV2B0 загружена и 'заморожена'.")

x_head = base_model.output
x_head = GlobalAveragePooling2D()(x_head)
x_head = Dense(256, activation='relu')(x_head)
x_head = Dropout(0.5)(x_head)
outputs = Dense(NUM_CLASSES, activation='softmax')(x_head)

model = Model(inputs=inputs, outputs=outputs)

low_lr_adam = Adam(learning_rate=0.0001)
model.compile(
    optimizer=low_lr_adam,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
print("\n---  Модель 'model' успешно собрана и скомпилирована (с Low LR)! ---")
model.summary()

--- Шаг 3 (ИСПРАВЛЕННЫЙ, Low LR): Построение Unimodal Модели ---
Форма входа: (224, 224, 3), Классов: 7
24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Базовая модель EfficientNetV2B0 загружена и 'заморожена'.

---  Модель 'model' успешно собрана и скомпилирована (с Low LR)! ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 224, 224,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 224, 224,  │          0 │ rescaling[0][0]   │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 224, 224,  │          0 │ rescaling_1[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ normalization[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │      4,608 │ stem_activation[… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 112, 112,  │         64 │ block1a_project_… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_ac… │ (None, 112, 112,  │          0 │ block1a_project_… │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_conv │ (None, 56, 56,    │      9,216 │ block1a_project_… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_bn   │ (None, 56, 56,    │        256 │ block2a_expand_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_expand_act… │ (None, 56, 56,    │          0 │ block2a_expand_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_co… │ (None, 56, 56,    │      2,048 │ block2a_expand_a… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2a_project_bn  │ (None, 56, 56,    │        128 │ block2a_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_conv │ (None, 56, 56,    │     36,864 │ block2a_project_… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2b_expand_bn   │ (None, 56, 56,    │        512 │ block2b_expand_c

 Total params: 6,249,047 (23.84 MB)

 Trainable params: 329,735 (1.26 MB)

 Non-trainable params: 5,919,312 (22.58 MB)

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

print("--- Шаг 4: Запуск Обучения Модели ---")

checkpoint_cb = ModelCheckpoint(
    'best_skin_cancer_model.keras',
    monitor='val_accuracy',
    save_best_only=True,
    mode='max'
)

early_stopping_cb = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    mode='min'
)

EPOCHS = 15

print(f"Начинаем обучение на {EPOCHS} эпох...")

try:
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=val_dataset,
        callbacks=[checkpoint_cb, early_stopping_cb]
    )
    print("\n---  Обучение (Шаг 4) Завершено! ---")

except NameError as e:
    print(f"ОШИБКА: {e}")
    print("Убедитесь, что 'model', 'train_dataset' или 'val_dataset' существуют.")
    print("Пожалуйста, сначала выполните предыдущие ячейки (Шаги 2 и 3).")

except Exception as e:
    print(f"Произошла ошибка во время обучения: {e}")
    raise

--- Шаг 4: Запуск Обучения Модели ---
Начинаем обучение на 15 эпох...
Epoch 1/15
276/276 ━━━━━━━━━━━━━━━━━━━━ 117s 294ms/step - accuracy: 0.2409 - loss: 1.7306 - val_accuracy: 0.2453 - val_loss: 1.6540
Epoch 2/15
276/276 ━━━━━━━━━━━━━━━━━━━━ 48s 163ms/step - accuracy: 0.2325 - loss: 1.6828 - val_accuracy: 0.2390 - val_loss: 1.6537
Epoch 3/15
276/276 ━━━━━━━━━━━━━━━━━━━━ 44s 148ms/step - accuracy: 0.2346 - loss: 1.6801 - val_accuracy: 0.2390 - val_loss: 1.6505
Epoch 4/15
276/276 ━━━━━━━━━━━━━━━━━━━━ 44s 148ms/step - accuracy: 0.2441 - loss: 1.6750 - val_accuracy: 0.2453 - val_loss: 1.6493
Epoch 5/15
276/276 ━━━━━━━━━━━━━━━━━━━━ 85s 159ms/step - accuracy: 0.2574 - loss: 1.6707 - val_accuracy: 0.2417 - val_loss: 1.6495
Epoch 6/15
276/276 ━━━━━━━━━━━━━━━━━━━━ 45s 153ms/step - accuracy: 0.2502 - loss: 1.6703 - val_accuracy: 0.2390 - val_loss: 1.6498
Epoch 7/15
276/276 ━━━━━━━━━━━━━━━━━━━━ 46s 155ms/step - accuracy: 0.2409 - loss: 1.6742 - val_accuracy: 0.2390 - val_loss: 1.6502

---  Обучен

In [ ]:
import tensorflow as tf
import os

print("--- Шаг 2 (Новый): Создание DataLoader'а из папок ---")

IMG_SIZE = 224
BATCH_SIZE = 32

BASE_DIR = 'dataset_resampled'
TRAIN_DIR = os.path.join(BASE_DIR, 'Resampled_HAM10000', 'Training')
TEST_DIR = os.path.join(BASE_DIR, 'Resampled_HAM10000', 'Testing')

if not os.path.exists(TRAIN_DIR):
    print(f"ОШИБКА: Папка Training не найдена по пути: {TRAIN_DIR}")
    print("Убедись, что 'Resampled_HAM10000.zip' был распакован правильно.")
    raise FileNotFoundError

train_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_DIR,
    shuffle=False,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
)

class_names = train_dataset.class_names
NUM_CLASSES = len(class_names)

print(f"\nНайдено классов: {NUM_CLASSES}")
print(f"Имена классов: {class_names}")

train_dataset = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print("\n---  'train_dataset', 'val_dataset' и 'test_dataset' созданы! ---")
print("Загрузчики данных (из Resampled набора) готовы к обучению.")

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Dropout, Rescaling
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetV2B0

print("--- Шаг 3: Построение Unimodal Модели (EfficientNetV2) ---")

try:
    input_shape = (IMG_SIZE, IMG_SIZE, 3)
    print(f"Форма входа: {input_shape}")
    print(f"Количество классов: {NUM_CLASSES}")
except NameError:
    print("ОШИБКА: Переменные 'IMG_SIZE' или 'NUM_CLASSES' не найдены.")
    print("Пожалуйста, сначала выполните предыдущую ячейку (Шаг 2).")
    raise

inputs = Input(shape=input_shape)

x = Rescaling(1./255)(inputs)

# --- 3. Загрузка БАЗОВОЙ Модели (EfficientNetV2) ---
# include_top=False означает, что мы не загружаем финальный классификатор ImageNet
base_model = EfficientNetV2B0(
    include_top=False,
    weights='imagenet', # Загружаем веса, обученные на ImageNet
    input_tensor=x # Подключаем к нашему 'x'
)

# "Замораживаем" веса базовой модели. Мы не будем их обучать (пока).
base_model.trainable = False
print("Базовая модель EfficientNetV2B0 загружена и 'заморожена'.")

# --- 4. Создание "ГОЛОВЫ" (Наш кастомный классификатор) ---

# "Сплющиваем" выход из базовой модели
x_head = base_model.output
x_head = GlobalAveragePooling2D()(x_head)

# Добавляем наш собственный плотный слой для обработки признаков
x_head = Dense(256, activation='relu')(x_head)

# Добавляем Dropout для предотвращения переобучения
x_head = Dropout(0.5)(x_head)

# Финальный выходной слой
# 'softmax' используется для мульти-классовой классификации
outputs = Dense(NUM_CLASSES, activation='softmax')(x_head)

# --- 5. Сборка Модели ---
model = Model(inputs=inputs, outputs=outputs)

# --- 6. Компиляция Модели ---
# 'sparse_categorical_crossentropy' используется, потому что
# tf.keras.utils.image_dataset_from_directory автоматически создает
# метки (Y) в виде чисел (0, 1, 2...)
model.compile(
    optimizer='adam', # 'Adam' - стандартный и надежный оптимизатор
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("\n--- ✅ Модель 'model' успешно собрана и скомпилирована! ---")

# --- 7. Просмотр Архитектуры ---
print("Архитектура модели:")
model.summary()

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

print("--- Шаг 4: Запуск Обучения Модели ---")

# --- 1. Определение Callbacks ---

# Сохраняем лучшую модель (по val_accuracy)
checkpoint_cb = ModelCheckpoint(
    'best_model_resampled.keras',  # Имя файла для сохранения
    monitor='val_accuracy',
    save_best_only=True,
    mode='max' # 'max' потому что мы следим за 'accuracy'
)

# Останавливаем обучение, если val_loss не улучшается
early_stopping_cb = EarlyStopping(
    monitor='val_loss',
    patience=5, # Ждем 5 эпох без улучшения (даем модели больше шансов)
    restore_best_weights=True, # Восстанавливаем лучшие веса в конце
    mode='min' # 'min' потому что мы следим за 'loss'
)

# --- 2. Определение Эпох ---
EPOCHS = 20 # Начнем с 20 эпох

# --- 3. Запуск Обучения (model.fit) ---
print(f"Начинаем обучение на {EPOCHS} эпох...")

# Убедимся, что 'model', 'train_dataset', 'val_dataset' в памяти
try:
    history = model.fit(
        train_dataset,
        epochs=EPOCHS,
        validation_data=val_dataset,
        callbacks=[checkpoint_cb, early_stopping_cb] # Подключаем "помощников"
    )
    print("\n--- ✅ Обучение (Шаг 4) Завершено! ---")

except NameError as e:
    print(f"ОШИБКА: {e}")
    print("Убедитесь, что 'model', 'train_dataset' или 'val_dataset' существуют.")
    print("Пожалуйста, сначала выполните предыдущие ячейки (Шаги 2 и 3).")

except Exception as e:
    print(f"Произошла ошибка во время обучения: {e}")
    raise

In [ ]:
import os

print("--- 🕵️‍♂️ ДЕБАГ: Проверка Путей к Данным ---")

BASE_DIR = 'dataset_resampled'
# --- !!! ВОЗМОЖНО, ПУТЬ НЕПРАВИЛЬНЫЙ !!! ---
# Мы предположили, что zip распакуется в 'dataset_resampled/Resampled_HAM10000/...'
# Давай проверим это.
TRAIN_DIR = os.path.join(BASE_DIR, 'Resampled_HAM10000', 'Training')

# Проверяем, существует ли сама папка 'dataset_resampled'
if not os.path.exists(BASE_DIR):
    print(f"ОШИБКА: Базовая папка '{BASE_DIR}' не найдена!")
    print("Убедись, что ты правильно распаковал zip-архив.")
else:
    print(f"Базовая папка '{BASE_DIR}' найдена.")

# Проверяем, существует ли папка 'Training'
if not os.path.exists(TRAIN_DIR):
    print(f"ОШИБКА: Папка 'Training' не найдена по пути: {TRAIN_DIR}")
    print("Проверь структуру папок в 'dataset_resampled'.")
else:
    print(f"Папка 'Training' ({TRAIN_DIR}) найдена.")

    # --- Самая важная проверка ---
    # Считаем, сколько файлов в подпапках
    try:
        total_files = 0
        subdirs = os.listdir(TRAIN_DIR)
        print(f"Найдены подпапки (классы): {subdirs}")

        for subdir in subdirs:
            if subdir.startswith('.'): continue # Игнорируем .ipynb_checkpoints

            subdir_path = os.path.join(TRAIN_DIR, subdir)
            if os.path.isdir(subdir_path):
                num_files = len(os.listdir(subdir_path))
                print(f"  - В папке '{subdir}': {num_files} изображений")
                total_files += num_files

        print(f"\n--- ИТОГО ФАЙЛОВ В 'Training': {total_files} ---")

        if total_files < 1000:
            print("\n--- ❗️❗️❗️ ПРЕДУПРЕЖДЕНИЕ: ОЧЕНЬ МАЛО ФАЙЛОВ! ---")
            print("Это объясняет, почему обучение провалилось (16/16 батчей).")
        else:
            print(f"\n--- ✅ ОТЛИЧНО: Найдено {total_files} файлов. ---")

    except Exception as e:
        print(f"Не удалось просканировать папку 'Training': {e}")